In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.9 MB/s eta 0:00:00


In [2]:
!pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 2.9 MB/s eta 0:00:00


In [92]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
# from plot_metric.functions import BinaryClassification

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [157]:
import json
data = {
  "account": "",
  "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
  "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
  "quota_project_id": "dev-sd-lake",
  "refresh_token": "1//05nz9IRnR1e2JCgYIARAAGAUSNwF-L9Ir_zi7EkfNFJb02e5XxL3e4Qicf7BN708CjMo66DuPPMhkd09jS6lIra-zubUPFGORcyU",
  "type": "authorized_user",
  "universe_domain": "googleapis.com"
}
json_data = json.dumps(data)
!echo '{json_data}' > '/content/.config/application_default_credentials.json'

In [158]:
from google.cloud import bigquery

client = bigquery.Client(project='dev-sd-lake')

product_listings_query = """
SELECT * FROM pollen.product_listings;
"""

product_listings = client.query_and_wait(product_listings_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

products_query = """
SELECT * FROM pollen.products;
"""

products = client.query_and_wait(products_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_categories_query = """
SELECT * FROM pollen.product_categories;
"""

product_categories = client.query_and_wait(product_categories_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_subcategories_query = """
SELECT * FROM pollen.product_subcategories;
"""

product_subcategories = client.query_and_wait(product_subcategories_query).to_dataframe()

client = bigquery.Client(project='dev-sd-lake')

sellers_query = """
SELECT * FROM pollen.sellers;
"""

sellers = client.query_and_wait(sellers_query).to_dataframe()

In [159]:
offers_query = """
SELECT * FROM pollen.offers;
"""

offers = client.query_and_wait(offers_query).to_dataframe()

In [160]:
orders_level_1_query = """
SELECT * FROM pollen.orders_level_1;
"""

orders_level_1 = client.query_and_wait(orders_level_1_query).to_dataframe()

In [161]:
orders_level_2_query = """
SELECT * FROM pollen.orders_level_2;
"""

orders_level_2 = client.query_and_wait(orders_level_2_query).to_dataframe()

In [162]:
sellers = sellers[['seller_name', 'persona_seller_type']].drop_duplicates()

In [163]:
print(product_listings.shape, products.shape, product_categories.shape, product_subcategories.shape, sellers.shape)

(99164, 41) (99172, 9) (22, 3) (111, 4) (109, 2)


In [164]:
def get_order_date(df_order_sheet1, df_order_sheet2):
  order_date_to_tracking_no_mapping = df_order_sheet1[['tracking_no', 'order_date']].drop_duplicates()
  df_order_sheet2 = pd.merge(df_order_sheet2, order_date_to_tracking_no_mapping, 'left',  on = ['tracking_no'])
  df_order_sheet2.drop('date_of_order', axis = 1, inplace = True)
  df_order = df_order_sheet2.copy()
  return df_order

orders = get_order_date(orders_level_1, orders_level_2)

In [165]:
orders = orders.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.rename(columns = {'total_units' : 'total_units_offered'})
orders = orders.rename(columns = {'total_units' : 'total_units_ordered', 'total_offer_price_usd' : 'total_order_price_usd'})

offers = pd.merge(offers, orders[['sku_number', 'expiry_date', 'tracking_no', 'total_units_ordered', 'recovery_rate_percentage', 'total_order_price_usd']], 'left', on = ['sku_number', 'expiry_date', 'tracking_no'])

In [166]:
product_subcategories = product_subcategories.rename(columns = {'id' : 'subcategory_id', 'name' : 'product_subcategory'})
product_categories = product_categories.rename(columns = {'id' : 'category_id', 'name' : 'product_category'})

products = pd.merge(products, product_categories, 'left', on = 'category_id')
products = pd.merge(products, product_subcategories[['subcategory_id', 'product_subcategory']], 'left', on = 'subcategory_id')

In [167]:
products = products.rename(columns = {'id' : 'product_id'})
products = products.drop(['parent_product_id', 'created_at', 'updated_at', 'category_id', 'subcategory_id'], axis = 1)

product_listings = product_listings.drop(['created_at', 'updated_at', 'pollen_updated_price_per_unit_local',
                       'barcode', 'barcode_key', 'manufactured_date', 'batch_number', 'image_links', 'usd_conversion', 'scoring'], axis = 1)
product_listings = pd.merge(product_listings, products, 'left', on = 'product_id')

In [168]:
product_listings.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,7c765b8d-4858-42f3-bdcd-d39e07146f58,7468eef6-5131-48d9-89d3-e13006c70311,SE1712631761502,SE1712631761502G4311200,L'Oreal Indonesia,2025-01-01,Carton,64.20,60.0,3852.0,IDR,24900.0,5991.5,31.0,g,0.01,254.0,156.0,219.0,MM,2.24,Bekasi,Indonesia,-,-,NaN,P2,1.53,0.37,CORE,2024-05-28,G4311200,GRN 2 STEP PW MASK B5 GOJI EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS
1,b419d6ba-dd0c-4283-83de-df3a38e7a9be,9eb95253-892b-4eea-9fa0-33a79786d3d1,SE1712631761502,SE1712631761502G4218000,L'Oreal Indonesia,2025-01-28,Carton,58.43,60.0,3506.0,IDR,23400.0,6552.7,19.7,g,0.01,185.0,150.0,197.0,MM,1.42,Bekasi,Indonesia,-,-,NaN,P2,1.44,0.40,CORE,2024-05-28,G4218000,GRN 5 MIN AMP MASK WATERMELON EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS


In [169]:
product_listings.inventory_class.value_counts()

,count
inventory_class,
CORE,66703
NON_CORE,32461


In [170]:
df_inv = product_listings[product_listings.inventory_class == 'CORE']
non_core_inv = product_listings[product_listings.inventory_class == 'NON_CORE']
df_inv = df_inv.reset_index(drop = True)
non_core_inv = non_core_inv.reset_index(drop = True)

In [171]:
cols_list_underscore_cleaning = ['sku_product_name', 'brand', 'product_category', 'product_subcategory', 'warehouse_country',
            'country_of_origin', 'pack_label_language', 'product_restricted_countries', 'shelf_life_bucket']

inv_cols_list_to_change_dtypes = ['qty_of_cartons', 'units_per_cartons', 'retail_price_per_case_(local)', 'retail_price_per_case_(usd)', 'asking_price_per_case_(local)',
                              'asking_price_per_case_(usd)', 'total_retail_price_(local)', 'total_retail_price_(usd)', 'total_asking_price_(local)',
                              'total_asking_price_(usd)', 'pack_size_(number)', 'total_cbm', 'cbm_per_case', 'package_dimensions_per_package_type_(length)',
                              'package_dimensions_per_case_(width)', 'package_dimensions_per_case_(height)', 'net_weight_\nper_unit(kg)', 'net_weight_per_case_(kg)',
                              'gross_weight_per_case_(kg)', 'total_net_weight_(kg)', 'total_gross_weight_(kg)', 'cases_per_pallet', 'number_of_pallets', 'discount',
                              'shelf_remaining_days']

record_attributes = ['sku_number', 'sku_name', 'brand', 'product_category', 'product_subcategory', 'retail_price_per_unit_local', 'currency']

n_days_before_date = pd.to_datetime(date.today() - timedelta(days = 14))

THRESHOLD = 0.65

In [172]:
def dropping_null_brand_cat_subcats(df_inv):
  df_inv = df_inv[~df_inv.brand.isnull()]
  df_inv = df_inv[~df_inv.product_category.isnull()]
  df_inv = df_inv[~df_inv.product_subcategory.isnull()]
  return df_inv

def cols_to_lower_rem_space(df, cols_list):
  for col in ['sku_name', 'product_name', 'brand', 'brand_', 'product_description', 'product_category', 'product_sub_category', 'product_subcategory',\
              'warehouse_location', 'buyer', 'seller', 'package_type', 'deal_type', 'origin_', 'destination', 'region_of_export', \
              'domestic_export', 'order_type', 'country', 'currency', 'country_of_origin', 'pack_label_language', 'dangerous_goods\n(y/n)', 'shelf_life_bucket',
              'relavant_(r)/ir-relavant(ir)', 'product_restricted_countries', 'store_name', 'quarter', 'temp_reference', 'tracking_no', 'offer_type', 'fiscal_month',
              'proposed_buyer_product_preferences', 'product_shelf_life_months', 'priority', 'seller', 'lms_seller_id', 'persona', 'core_vs._\nnon_core', 'sku_product_name',
              'warehouse_country', 'warehouse_address', 'measurement_units', 'listing_currency', 'manufacturing_country', 'lbh_measurement_units', 'package_type', 'seller_name',
              'pack_size_unit', 'package_labeled_language', 'inventory_class', 'description', 'unit_dimensions']:
              if col in cols_list:
                df[col] = df[col].str.lower().str.replace(' ', '_')
  return df

def remove_starting_and_trailing_underscores(x):
  if x != '':
    if x[0] == '_':
      for i in range(len(x)):
        if x[i] != '_':
          start = i
          break
      x = x[start:]
    if x[-1] == '_':
      for i in reversed(range(len(x))):
        if x[i] != '_':
          end = i
          break
      x = x[:end+1]
  return x

def cleaning_underscores_inv_data(df_inv, col_list):
  for col in col_list:
    print(col)
    df_inv[col] = df_inv[col].apply(remove_starting_and_trailing_underscores)
  return df_inv

def drop_dups_and_clean(df_inv):
  print('Dropping above duplicates ...')
  df_inv = df_inv.drop_duplicates()
  return df_inv

In [173]:
df_inv = dropping_null_brand_cat_subcats(df_inv)
df_inv = cols_to_lower_rem_space(df_inv, df_inv.columns)
df_inv = drop_dups_and_clean(df_inv)

non_core_inv = cols_to_lower_rem_space(non_core_inv, non_core_inv.columns)

Dropping above duplicates ...


# Non Core

In [174]:
non_core_inv.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,2d28d351-f531-42d7-abc1-fee72e225c53,1e18e385-f133-4fb3-95f4-b56cb1419369,SE1712631761542,SE171263176154204R3I1MKJPD97H7OBZ60UTU1Z,acommerce_singapore,2023-05-31,pieces,NaN,NaN,6.0,sgd,19.9,1.99,NaN,None,0.0,10.0,10.0,10.0,cm,0.50,sg-acom01,None,None,None,NaN,p1,15.116504,1.511650,non_core,2023-01-25,04R3I1MKJPD97H7OBZ60UTU1Z,l'oreal_paris_color_riche_satin_105_nude_in_paris,l'oreal_paris,makeup,makeup,lip_makeup
1,063c43ca-8297-4bea-9bbf-3ff2e9fd8376,3a7331cf-dbd4-4cb4-b791-982833ff97ff,SE1712631761542,SE171263176154211YAP8LR1W3OQSFJGWCLTD1DQ,acommerce_singapore,2023-05-31,pieces,NaN,NaN,160.0,sgd,2.0,0.20,NaN,None,0.0,8.0,5.0,0.2,cm,0.15,sg-acom02,None,None,None,NaN,p1,1.519247,0.151925,non_core,2023-01-25,11YAP8LR1W3OQSFJGWCLTD1DQ,garnier_serum_mask_-_pomegranate_(replumping),garnier,skin_care,skin_care,masks_&_exfoliators


In [175]:
non_core_inv = non_core_inv.rename(columns = {'total_number_of_items' : 'total_units'})
non_core_inv['updated_on'] = pd.to_datetime(non_core_inv['updated_on'])
non_core_inv['expiry_date'] = pd.to_datetime(non_core_inv['expiry_date'])

In [176]:
print(non_core_inv.shape)
non_core_inv = non_core_inv.drop_duplicates(subset = ['sku_number', 'expiry_date', 'updated_on', 'seller_name'])
print(non_core_inv.shape)

(32461, 37)
(29476, 37)


In [177]:
offers = offers.rename(columns = {'seller' : 'seller_name'})
offers['date_of_offer'] = pd.to_datetime(offers['date_of_offer'])
offers['seller_name'] = offers['seller_name'].str.lower()
offers['total_units_ordered'] = offers['total_units_ordered'].str.replace(',', '')
offers['total_units_ordered'] = offers['total_units_ordered'].astype(float)

In [178]:
def merge_non_core_inv_and_offer(non_core_inv, df_offer):
  temp = pd.merge(non_core_inv, df_offer[['sku_number', 'seller_name', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                          'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd']], 'left', on = ['sku_number', 'seller_name'])
  # temp['updated_on'] = pd.to_datetime(temp['updated_on'])
  temp['offer_day_diff_updated_inv'] = (temp['date_of_offer'] - temp['updated_on']).dt.days
  # avoiding future entry from same sku and seller to get filtered out hence nulling it as will remain in the dataframe
  temp['offer_day_diff_updated_inv'] = np.where(temp.offer_day_diff_updated_inv < 0, np.nan, temp['offer_day_diff_updated_inv'])
  temp = temp[(temp.offer_day_diff_updated_inv >= 0) | (temp.offer_day_diff_updated_inv.isnull())]
  df_non_core_inv_min_offer_day_diff_updated_inv = temp.groupby(['sku_number', 'seller_name', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                        .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  temp = pd.merge(temp, df_non_core_inv_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'seller_name', 'date_of_offer'])
  temp = temp[(temp.offer_day_diff_updated_inv == temp.min_offer_day_diff_updated_inv) | (temp.offer_day_diff_updated_inv.isnull())]
  temp.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return temp

In [179]:
non_core_inv = merge_non_core_inv_and_offer(non_core_inv, offers)
# non_core_inv['sell_thru_rate'] = non_core_inv['total_units_ordered'] / non_core_inv['total_units']
# non_core_inv['sell_thru_rate'] = np.where(non_core_inv['sell_thru_rate'] > 1, 1, non_core_inv['sell_thru_rate'])

In [180]:
non_core_inv['shelf_life_remaining_days'] = (non_core_inv['expiry_date'] - non_core_inv['updated_on']).dt.days

In [181]:
temp = non_core_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units', 'shelf_life_remaining_days',
                     'warehouse_country', 'total_units_ordered', 'recovery_rate_percentage']].sort_values(by = ['sku_number', 'brand', 'product_category', 'product_subcategory',
                      'expiry_date', 'seller_name', 'updated_on', 'priority', 'warehouse_country'])
temp = temp.reset_index(drop = True)

In [182]:
def get_prev_inv_occurence_next_to_current_for_non_core_inv(temp):
  new_col_names_for_shifted_inv = dict()
  for i in temp.columns:
    new_col_names_for_shifted_inv[i] = i+'_prev'

  temp_shifted = temp.shift(1)
  temp_shifted = temp_shifted.rename(columns = new_col_names_for_shifted_inv)
  temp = pd.concat([temp, temp_shifted], axis=1)
  return temp

# not taking expiry into account

def get_repeated_inv_and_mean_depletion_for_non_core_inv(temp):
  repeated_inv = temp[(temp.sku_number == temp.sku_number_prev) & (temp.seller_name == temp.seller_name_prev)]
  repeated_inv['depletion'] = repeated_inv['total_units_prev'] - repeated_inv['total_units']
  repeated_inv['time'] = (repeated_inv['updated_on'] - repeated_inv['updated_on_prev']).dt.days
  repeated_inv = repeated_inv.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', \
                            'seller_name', 'updated_on', 'priority'])[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on',\
                                                                        'priority', 'depletion', 'time', 'total_units_prev', 'total_units', 'priority_prev',\
                                                                        'shelf_life_remaining_days_prev', 'warehouse_country_prev', 'total_units_ordered_prev',
                                                                        'recovery_rate_percentage_prev']]

  return repeated_inv

## Recommendation - measure depletion at some other level, exclude skus as many skus don't repeat and many rows are getting dropped

In [183]:
temp = get_prev_inv_occurence_next_to_current_for_non_core_inv(temp)
repeated_inv_non_core = get_repeated_inv_and_mean_depletion_for_non_core_inv(temp)
repeated_inv_non_core = repeated_inv_non_core.reset_index(drop = True)
repeated_inv_non_core['priority'] = np.where(repeated_inv_non_core.priority != repeated_inv_non_core.priority_prev, \
                                                   repeated_inv_non_core.priority_prev, repeated_inv_non_core.priority)

In [184]:
# 90 days sales life cycle
repeated_inv_non_core = repeated_inv_non_core[repeated_inv_non_core.time <= 90]

mean_depletion_per_cat_non_core_inv = repeated_inv_non_core.groupby(['product_category']).depletion.mean().reset_index().sort_values(by = 'depletion')

low_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(0.33)].product_category)
mid_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(0.67)].product_category)
mid_depletion_cats = [i for i in mid_depletion_cats if i not in low_depletion_cats]
high_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(1)].product_category)
high_depletion_cats = [i for i in high_depletion_cats if i not in low_depletion_cats + mid_depletion_cats]

print(low_depletion_cats, '\n', mid_depletion_cats, '\n', high_depletion_cats)

['beverages', 'health_&_supplments', 'industrial', 'oral_care', 'skin_care'] 
 ['baby', 'hair_care', 'home_care', 'fashion_-_accessories'] 
 ['fragrance', 'sports,_fitness,_&_outdoors', 'makeup', 'home', 'food']


In [185]:
def set_depletion_non_core(row):
  if row['depletion'] < 0:
    return row['total_units_prev']
  elif row['product_category'] in high_depletion_cats and row['time'] <= 20:
    return row['depletion']
  elif row['product_category'] in high_depletion_cats and row['time'] > 20:
    return row['total_units_prev']
  elif row['product_category'] in mid_depletion_cats and row['time'] <= 45:
    return row['depletion']
  elif row['product_category'] in mid_depletion_cats and row['time'] > 45:
    return row['total_units_prev']
  elif row['product_category'] in low_depletion_cats and row['time'] <= 60:
    return row['depletion']
  elif row['product_category'] in low_depletion_cats and row['time'] > 60:
    return row['total_units_prev']

In [186]:
repeated_inv_non_core['depletion'] = repeated_inv_non_core.apply(set_depletion_non_core, axis=1)

In [187]:
def remove_percent_sign(x):
  if pd.isna(x):
    return x
  else:
    return x[:-1]
repeated_inv_non_core['recovery_rate_percentage_prev'] = repeated_inv_non_core['recovery_rate_percentage_prev'].apply(remove_percent_sign)
repeated_inv_non_core['recovery_rate_percentage_prev'] = np.where(repeated_inv_non_core['recovery_rate_percentage_prev'] == '', np.nan, repeated_inv_non_core['recovery_rate_percentage_prev'])
repeated_inv_non_core['recovery_rate_percentage_prev'] = repeated_inv_non_core['recovery_rate_percentage_prev'].astype(float)
repeated_inv_non_core['recovery_rate_percentage_prev'] = np.where(repeated_inv_non_core['recovery_rate_percentage_prev'].isnull(), 0 , repeated_inv_non_core['recovery_rate_percentage_prev'])
repeated_inv_non_core['total_units_ordered_prev'] = np.where(repeated_inv_non_core['total_units_ordered_prev'].isnull(), 0, repeated_inv_non_core['total_units_ordered_prev'])

In [188]:
repeated_inv_non_core = repeated_inv_non_core.rename(columns = {'total_units_ordered_prev' : 'pollen_liquidated_units',
                                                                'recovery_rate_percentage_prev' : 'pollen_recovery_rate_percentage'})

In [189]:
repeated_inv_non_core.sample(2)

,sku_number,brand,product_category,product_subcategory,expiry_date,seller_name,updated_on,priority,depletion,time,total_units_prev,total_units,priority_prev,shelf_life_remaining_days_prev,warehouse_country_prev,pollen_liquidated_units,pollen_recovery_rate_percentage
112,2800-3BN,marmolux_acc,home,home_-_other,NaT,rainforest,2023-10-30,p1,1.0,47,1.0,120.0,p1,NaN,None,0.0,0.0
313,3CEID012511,3_ce,makeup,makeup_accessories,NaT,eblo,2024-06-13,p1,59.0,56,59.0,9.0,p1,NaN,None,0.0,0.0


# Core

In [190]:
offers = offers.rename(columns = {'seller_name' : 'seller_short'})

seller_name_to_short_form = {'unilever_indonesia' : 'ulid', 'unilever_malaysia' : 'ulmy', 'unilever_philippines' : 'ulph', 'unilever_thailand' : 'ulth', 'unilever_singapore' : 'ulsg',
                             "l'oreal_malaysia" : "lomy", "l'oreal_philippines" : 'loph', "l'oreal_thailand" : "loth", "l'oreal_india" : "loin", "l'oreal_indonesia" : "loid"}

df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])

def apply_seller_short(x):
  if x in seller_name_to_short_form.keys():
    return seller_name_to_short_form[x]
  else:
    return x

df_inv['seller_short'] = df_inv['seller_name'].apply(apply_seller_short)

offers = offers[offers['expiry_date'] != '-']
offers['expiry_date'] = pd.to_datetime(offers['expiry_date'], format = "%d-%b-%Y")
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])

In [191]:
def merge_inv_and_offer(df_inv, df_offer):
  df_inv = pd.merge(df_inv, df_offer[['sku_number', 'expiry_date', 'seller_short', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                      'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd']], 'left', on = ['sku_number', 'expiry_date', 'seller_short'])
  df_inv['offer_day_diff_updated_inv'] = (df_inv['date_of_offer'] - df_inv['updated_on']).dt.days
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv >= 0) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_min_offer_day_diff_updated_inv = df_inv.groupby(['sku_number', 'expiry_date', 'seller_short', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                      .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  df_inv = pd.merge(df_inv, df_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'expiry_date', 'seller_short', 'date_of_offer'])
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv == df_inv.min_offer_day_diff_updated_inv) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_inv.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return df_inv

In [192]:
df_inv = merge_inv_and_offer(df_inv, offers)
df_inv['total_units_ordered'] = df_inv['total_units_ordered'].fillna(0)
# df_inv['sell_thru_rate'] = df_inv['total_units_ordered'] / df_inv['total_number_of_items']
# df_inv['sell_thru_rate'] = np.where(df_inv['sell_thru_rate'] > 1, 1, df_inv['sell_thru_rate'])
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

In [193]:
# dropping duplicates considering all columns except the ids
k = list(set(df_inv.columns) - set([i for i in df_inv.columns if i.endswith('id')]))
df_inv = df_inv.drop_duplicates(subset = k)

In [194]:
df_inv = df_inv.rename(columns = {'total_number_of_items' : 'total_units'})
df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])
df_inv = df_inv[df_inv.sku_number != '']
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

def remove_percent_sign(x):
  if pd.isna(x):
    return x
  else:
    return x[:-1]
df_inv['recovery_rate_percentage'] = df_inv['recovery_rate_percentage'].apply(remove_percent_sign)
df_inv['recovery_rate_percentage'] = np.where(df_inv['recovery_rate_percentage'] == '', np.nan, df_inv['recovery_rate_percentage'])
df_inv['recovery_rate_percentage'] = df_inv['recovery_rate_percentage'].astype(float)
df_inv['recovery_rate_percentage'] = np.where(df_inv['recovery_rate_percentage'].isnull(), 0 , df_inv['recovery_rate_percentage'])

In [195]:
key_to_total_units_ordered = df_inv.groupby(['sku_number', 'seller_name', 'expiry_date', 'updated_on']).agg({'total_units_ordered' : 'sum'}).reset_index().rename(columns =
                                                {'total_units_ordered' : 'total_units_ordered_sum'})
df_inv = pd.merge(df_inv, key_to_total_units_ordered, 'left', on = ['sku_number', 'seller_name', 'expiry_date', 'updated_on'])

In [196]:
temp = df_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units',
               'shelf_life_remaining_days', 'warehouse_country', 'total_units_ordered_sum', 'recovery_rate_percentage']]\
                  .sort_values(by = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'warehouse_country'])
temp = temp.reset_index(drop = True)

In [197]:
print(temp.shape)
print(temp.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units']).shape)

(54892, 13)
(51240, 13)


In [198]:
def get_prev_inv_occurence_next_to_current(temp):
  new_col_names_for_shifted_inv = dict()
  for i in temp.columns:
    new_col_names_for_shifted_inv[i] = i+'_prev'

  temp_shifted = temp.shift(1)
  temp_shifted = temp_shifted.rename(columns = new_col_names_for_shifted_inv)
  temp = pd.concat([temp, temp_shifted], axis=1)
  return temp

def get_repeated_inv_and_mean_depletion(temp):
  repeated_inv = temp[(temp.sku_number == temp.sku_number_prev) & (temp.expiry_date == temp.expiry_date_prev) & (temp.seller_name == temp.seller_name_prev)]
  repeated_inv['depletion'] = repeated_inv['total_units_prev'] - repeated_inv['total_units']
  repeated_inv['time'] = (repeated_inv['updated_on'] - repeated_inv['updated_on_prev']).dt.days
  repeated_inv = repeated_inv.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', \
                            'seller_name', 'updated_on', 'priority'])[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on',\
                                                                        'priority', 'depletion', 'time', 'total_units_prev', 'total_units', 'priority_prev',\
                                                                        'shelf_life_remaining_days_prev', 'warehouse_country_prev', 'total_units_ordered_sum_prev',
                                                                        'recovery_rate_percentage_prev']]
  return repeated_inv

In [199]:
temp = get_prev_inv_occurence_next_to_current(temp)
repeated_inv = get_repeated_inv_and_mean_depletion(temp)

# since current was match to previous, hence correcting the current row previous priority order to previous one
repeated_inv['priority'] = np.where(repeated_inv.priority != repeated_inv.priority_prev, repeated_inv.priority_prev, repeated_inv.priority)

In [200]:
print(repeated_inv[repeated_inv.time > 90].shape)
print('Removing above number of rows as sales cycle is fixed to 90 days max for any cat or sub cat after which it is assumed to finish')
repeated_inv = repeated_inv[repeated_inv.time <= 90]

(2899, 17)
Removing above number of rows as sales cycle is fixed to 90 days max for any cat or sub cat after which it is assumed to finish


In [201]:
mean_depletion_per_cat_core_inv = repeated_inv.groupby(['product_category']).depletion.mean().reset_index().sort_values(by = 'depletion')

In [202]:
low_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(0.33)].product_category)
mid_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(0.67)].product_category)
mid_depletion_cats = [i for i in mid_depletion_cats if i not in low_depletion_cats]
high_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(1)].product_category)
high_depletion_cats = [i for i in high_depletion_cats if i not in low_depletion_cats + mid_depletion_cats]

In [203]:
print(low_depletion_cats, '\n', mid_depletion_cats, '\n', high_depletion_cats)

['beverages', 'fashion_-_accessories', 'home', 'oral_care'] 
 ['makeup', 'fragrance', 'skin_care'] 
 ['baby', 'hair_care', 'home_care', 'food']


In [204]:
# do disposal correction

In [205]:
def set_depletion(row):
  if row['depletion'] < 0:
    return row['total_units_prev']
  elif row['product_category'] in high_depletion_cats and row['time'] <= 20:
    return row['depletion']
  elif row['product_category'] in high_depletion_cats and row['time'] > 20:
    return row['total_units_prev']
  elif row['product_category'] in mid_depletion_cats and row['time'] <= 45:
    return row['depletion']
  elif row['product_category'] in mid_depletion_cats and row['time'] > 45:
    return row['total_units_prev']
  elif row['product_category'] in low_depletion_cats and row['time'] <= 60:
    return row['depletion']
  elif row['product_category'] in low_depletion_cats and row['time'] > 60:
    return row['total_units_prev']

In [206]:
repeated_inv['depletion'] = repeated_inv.apply(set_depletion, axis=1)
repeated_inv = repeated_inv.reset_index(drop = True)

In [207]:
repeated_inv = repeated_inv.rename(columns = {'total_units_ordered_sum_prev' : 'pollen_liquidated_units', 'recovery_rate_percentage_prev' : 'pollen_recovery_rate_percentage'})

In [208]:
COLUMNS_FOR_DEPLETION_MODELING = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'time']

In [209]:
repeated_inv = repeated_inv[COLUMNS_FOR_DEPLETION_MODELING + ['priority', 'total_units_prev', 'depletion', 'shelf_life_remaining_days_prev', 'warehouse_country_prev',
                                                              'pollen_liquidated_units', 'pollen_recovery_rate_percentage']]
repeated_inv_non_core = repeated_inv_non_core[COLUMNS_FOR_DEPLETION_MODELING + ['priority', 'total_units_prev', 'depletion', 'shelf_life_remaining_days_prev', 'warehouse_country_prev',
                                                                                'pollen_liquidated_units', 'pollen_recovery_rate_percentage']]
print(repeated_inv.shape, repeated_inv_non_core.shape)

(19109, 13) (979, 13)


In [210]:
repeated_inv = pd.concat([repeated_inv, repeated_inv_non_core], ignore_index=True)
repeated_inv.depletion['depletion'] = np.where(repeated_inv.depletion < repeated_inv.pollen_liquidated_units, repeated_inv.pollen_liquidated_units, repeated_inv.depletion)
repeated_inv.shape

(20088, 13)

In [211]:
repeated_inv.sample(2)

,sku_number,brand,product_category,product_subcategory,seller_name,time,priority,total_units_prev,depletion,shelf_life_remaining_days_prev,warehouse_country_prev,pollen_liquidated_units,pollen_recovery_rate_percentage
8337,69729501,bango,food,cooking_ingredients,unilever_indonesia,0,p1,120.0,120.0,160.0,indonesia,0.0,0.00
18096,XTH00794,garnier,skin_care,body_lotions_&_creams,l'oreal_thailand,27,p2,2861.0,150.0,351.0,thailand,150.0,27.59


In [212]:
def correct_p1_depletion(df):
  if df['priority'] == 'p1' and np.random.uniform(low=1, high=99, size=(1,))[0] <= 80:
    return np.random.uniform(low=5, high=20, size=(1,))[0]/ 100 * df['depletion']
  else:
    return df['depletion']

repeated_inv['depletion'] = repeated_inv.apply(correct_p1_depletion, axis = 1)
repeated_inv['pollen_liquidated_units'] = np.where(repeated_inv.pollen_liquidated_units > repeated_inv.depletion, repeated_inv.depletion, repeated_inv.pollen_liquidated_units)
repeated_inv = repeated_inv.reset_index(drop = True)

In [213]:
high_time_dummy = repeated_inv.copy()

def get_dummy_high_time(df):
    if pd.isna(df['time']):
      return df['time']
    else:
      return np.random.uniform(
            low=df['time'],
            high=1.5 * df['time'], size=(1,))[0]

high_time_dummy['time'] = high_time_dummy.apply(get_dummy_high_time, axis = 1)
high_time_dummy['orig_time'] = repeated_inv['time']
high_time_dummy['time_factor'] = high_time_dummy['time'] / high_time_dummy['orig_time']
high_time_dummy['depletion'] = high_time_dummy['depletion'] * high_time_dummy['time_factor']
high_time_dummy['total_units_prev'] = np.where(high_time_dummy['total_units_prev'] < high_time_dummy['depletion'],
                                                    high_time_dummy['depletion'], high_time_dummy['total_units_prev'])

In [214]:
low_time_dummy = repeated_inv.copy()

def get_dummy_low_time(df):
    if pd.isna(df['time']):
      return df['time']
    else:
      return np.random.uniform(
            low=0.5 * df['time'],
            high=df['time'], size=(1,))[0]

low_time_dummy['time'] = low_time_dummy.apply(get_dummy_low_time, axis = 1)
low_time_dummy['orig_time'] = repeated_inv['time']
low_time_dummy['time_factor'] = low_time_dummy['time'] / low_time_dummy['orig_time']
low_time_dummy['depletion'] = low_time_dummy['depletion'] * low_time_dummy['time_factor']
low_time_dummy['total_units_prev'] = np.where(low_time_dummy['total_units_prev'] < low_time_dummy['depletion'],
                                                    low_time_dummy['depletion'], low_time_dummy['total_units_prev'])

In [215]:
high_shelf_life_dummy = repeated_inv.copy()

def get_dummy_high_shelf_life(df):
    if pd.isna(df['shelf_life_remaining_days_prev']):
      return df['shelf_life_remaining_days_prev']
    else:
      return np.random.uniform(
            low=df['shelf_life_remaining_days_prev'],
            high=1.5 * df['shelf_life_remaining_days_prev'], size=(1,))[0]

high_shelf_life_dummy['shelf_life_remaining_days_prev'] = high_shelf_life_dummy.apply(get_dummy_high_shelf_life, axis = 1)
high_shelf_life_dummy['orig_shelf_life'] = repeated_inv['shelf_life_remaining_days_prev']
high_shelf_life_dummy['shelf_life_factor'] = high_shelf_life_dummy['shelf_life_remaining_days_prev'] / high_shelf_life_dummy['orig_shelf_life']
high_shelf_life_dummy['depletion'] = high_shelf_life_dummy['depletion'] * high_shelf_life_dummy['shelf_life_factor']
high_shelf_life_dummy['total_units_prev'] = np.where(high_shelf_life_dummy['total_units_prev'] < high_shelf_life_dummy['depletion'],
                                                    high_shelf_life_dummy['depletion'], high_shelf_life_dummy['total_units_prev'])

In [216]:
low_shelf_life_dummy = repeated_inv.copy()

def get_dummy_low_shelf_life(df):
    if pd.isna(df['shelf_life_remaining_days_prev']):
      return df['shelf_life_remaining_days_prev']
    else:
      return np.random.uniform(
            low=0.5 * df['shelf_life_remaining_days_prev'],
            high=df['shelf_life_remaining_days_prev'], size=(1,))[0]

low_shelf_life_dummy['shelf_life_remaining_days_prev'] = low_shelf_life_dummy.apply(get_dummy_low_shelf_life, axis = 1)
low_shelf_life_dummy['orig_shelf_life'] = repeated_inv['shelf_life_remaining_days_prev']
low_shelf_life_dummy['shelf_life_factor'] = low_shelf_life_dummy['shelf_life_remaining_days_prev'] / low_shelf_life_dummy['orig_shelf_life']
low_shelf_life_dummy['depletion'] = low_shelf_life_dummy['depletion'] * low_shelf_life_dummy['shelf_life_factor']
low_shelf_life_dummy['total_units_prev'] = np.where(low_shelf_life_dummy['total_units_prev'] < low_shelf_life_dummy['depletion'],
                                                    low_shelf_life_dummy['depletion'], low_shelf_life_dummy['total_units_prev'])

In [217]:
repeated_inv = pd.concat([repeated_inv, high_shelf_life_dummy, low_shelf_life_dummy, low_time_dummy, high_time_dummy], ignore_index=True)
repeated_inv = repeated_inv.drop_duplicates()
repeated_inv = repeated_inv.reset_index(drop = True)
repeated_inv = repeated_inv.drop(['orig_shelf_life', 'shelf_life_factor', 'orig_time', 'time_factor'], axis = 1)

In [218]:
# some duplicates because of nan shelf life

print(repeated_inv.duplicated().sum())
repeated_inv = repeated_inv.drop_duplicates()

20


## Listing type

In [219]:
def get_listing_type(df):
  if df['shelf_life_remaining_days_prev'] > 365*2:
    return 'fresh'
  elif df['shelf_life_remaining_days_prev'] > 365 and df['pollen_recovery_rate_percentage'] > 0 and df['pollen_recovery_rate_percentage'] < 10:
    return 'obsolete'
  else:
    return 'excess'

repeated_inv['listing_condition'] = repeated_inv.apply(get_listing_type, axis = 1)

number_of_rows_to_randomize = int(repeated_inv.shape[0]*0.05)

change = repeated_inv.sample(number_of_rows_to_randomize//2).index
repeated_inv.loc[change,'listing_condition'] = 'damaged'
change = repeated_inv.sample(number_of_rows_to_randomize//2).index
repeated_inv.loc[change,'listing_condition'] = 'made_to_order'

In [220]:
repeated_inv.listing_condition.value_counts()

,count
listing_condition,
excess,85760
fresh,5629
made_to_order,2409
damaged,2347
obsolete,225


## Liquidation channel

In [221]:
repeated_inv['depletion'] = np.where(repeated_inv['depletion'] < repeated_inv['pollen_liquidated_units'], repeated_inv['pollen_liquidated_units'], repeated_inv['depletion'])
repeated_inv['depletion_other_than_pollen'] = repeated_inv['depletion'] - repeated_inv['pollen_liquidated_units']
repeated_inv['liquidation_channel'] = [['friends/family', 'distributor', 'retailer', 'liquidator', 'disposal', 'pollen']]*len(repeated_inv)
repeated_inv = repeated_inv.explode('liquidation_channel').reset_index(drop = True)

In [222]:
def get_depleted_units_via_liquidation_channel(df):
  if df['priority'] == 'p1':
    if df['liquidation_channel'] == 'friends/family':
      return 0
    elif df['liquidation_channel'] == 'distributor':
      return 0.2 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.2 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.2 * 0.8 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.8 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

  if df['priority'] == 'p2':
    if df['liquidation_channel'] == 'friends/family':
      return 0.6 * 0.05 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'distributor':
      return 0.6 * 0.25 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.6 * 0.20 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.6 * 0.5 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.4 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

  if df['priority'] == 'p3':
    if df['liquidation_channel'] == 'friends/family':
      return 0.8 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'distributor':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.2 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

In [223]:
repeated_inv['final_depletion'] = repeated_inv.apply(get_depleted_units_via_liquidation_channel, axis = 1)
repeated_inv['depletion'] = repeated_inv['final_depletion']
repeated_inv = repeated_inv.drop(['pollen_liquidated_units', 'depletion_other_than_pollen', 'final_depletion', 'pollen_recovery_rate_percentage'], axis = 1)

In [224]:
# after above explode, the relationship between time and liquidation channel + time and depletion number + time and shelf_life_remaining_days_prev
# went for a toss. This is for the correction to keep the sensitivity intact

repeated_inv['channel_group'] = repeated_inv.index // len(['friends/family', 'distributor', 'retailer', 'liquidator', 'disposal', 'pollen'])
max_depletion_in_each_group = repeated_inv.groupby('channel_group').agg({'depletion' : 'max'}).reset_index().rename(columns = {'depletion' : 'max_depletion'})
repeated_inv = pd.merge(repeated_inv, max_depletion_in_each_group, 'left', on = 'channel_group')

In [225]:
def time_correction(df):
  if df['max_depletion'] == 0:
    return df['time']
  else:
    return df['time'] * df['depletion'] / df['max_depletion']

repeated_inv['time'] = repeated_inv.apply(time_correction, axis = 1)

def shelf_life_correction(df):
  if df['max_depletion'] == 0:
    return df['shelf_life_remaining_days_prev']
  else:
    return df['shelf_life_remaining_days_prev'] * df['depletion'] / df['max_depletion']

repeated_inv['shelf_life_remaining_days_prev'] = repeated_inv.apply(shelf_life_correction, axis = 1)
repeated_inv = repeated_inv.drop(['channel_group', 'max_depletion'], axis = 1)

In [226]:
sellers['seller_name'] = sellers['seller_name'].str.lower().str.replace(' ', '_')
sellers['persona_seller_type'] = sellers['persona_seller_type'].str.lower().str.replace(' ', '_')
repeated_inv = pd.merge(repeated_inv, sellers, 'left', on = 'seller_name')
print(repeated_inv.priority.isnull().sum(), repeated_inv.persona_seller_type.isnull().sum())

18492 144


In [227]:
repeated_inv['priority'] = np.where(repeated_inv.priority.isnull(), '', repeated_inv.priority)
repeated_inv['persona_seller_type'] = np.where(repeated_inv.persona_seller_type.isnull(), '', repeated_inv.persona_seller_type)

repeated_inv = repeated_inv[~(repeated_inv['total_units_prev'] == 0)]
repeated_inv['depletion_percent'] = repeated_inv['depletion'] / repeated_inv['total_units_prev']
repeated_inv = repeated_inv[~repeated_inv['depletion_percent'].isnull()]
repeated_inv['depletion_percent'] = repeated_inv['depletion_percent']*100

In [229]:
repeated_inv = repeated_inv.rename(columns = {'warehouse_country_prev' : 'warehouse_country', 'shelf_life_remaining_days_prev' : 'shelf_life_remaining_days'})

In [230]:
repeated_inv.shape

(541122, 15)

In [231]:
print(repeated_inv.shape)
print(repeated_inv[repeated_inv.time < 0].shape)
repeated_inv['time'] = np.where(repeated_inv.time < 0, -repeated_inv.time, repeated_inv.time)

(541122, 15)
(900, 15)


In [232]:
priority_to_shelf_life_median = repeated_inv.groupby('priority').shelf_life_remaining_days.median().reset_index()
priority_to_shelf_life_median = dict(zip(list(priority_to_shelf_life_median.priority), list(priority_to_shelf_life_median.shelf_life_remaining_days)))
priority_to_shelf_life_median

{'p1': 67.50688085975906, 'p2': 179.25, 'p3': 439.0}

In [233]:
def fill_na_shelf_life_remaining_days(df):
  if pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p1':
    return priority_to_shelf_life_median['p1']
  elif pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p2':
    return priority_to_shelf_life_median['p2']
  elif pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p3':
    return priority_to_shelf_life_median['p3']
  else:
    return df['shelf_life_remaining_days']

repeated_inv['shelf_life_remaining_days'] = repeated_inv.apply(fill_na_shelf_life_remaining_days, axis = 1)

In [238]:
repeated_inv.sample(2)

,sku_number,brand,product_category,product_subcategory,seller_name,time,priority,total_units_prev,depletion,shelf_life_remaining_days,warehouse_country,listing_condition,liquidation_channel,persona_seller_type,depletion_percent
72826,G3522700,maybelline,makeup,face_makeup,l'oreal_malaysia,25.833333,p3,6.0,1.200000,680.833333,malaysia,fresh,disposal,principal,20.00000
529101,G2849502,garnier,skin_care,sunscreen_&_spf_products,l'oreal_thailand,18.725630,p3,5495.0,38.568316,247.480028,thailand,excess,liquidator,principal,0.70188


In [2]:
repeated_inv.shape

NameError: name 'repeated_inv' is not defined

## model default 5 - depletion modeling

In [1]:
train_cols = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'shelf_life_remaining_days', 'time', 'listing_condition', 'liquidation_channel']
cat_cols = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'listing_condition', 'liquidation_channel']

print(repeated_inv[train_cols].duplicated().sum())
repeated_inv = repeated_inv.drop_duplicates(subset = train_cols)

NameError: name 'repeated_inv' is not defined

In [240]:
repeated_inv['depletion_percent'] = np.log1p(repeated_inv['depletion_percent'])

X_train, X_val, y_train, y_val = train_test_split(repeated_inv[train_cols], repeated_inv['depletion_percent'], test_size = 0.2, random_state = 7)

# antilogging for different prompts
repeated_inv['depletion_percent'] = np.expm1(repeated_inv['depletion_percent'])

In [241]:
depletion_model_params = {'loss_function': 'RMSE',
          'od_wait' : 25,
          'learning_rate': 0.6,
          'depth': 5,
          'min_data_in_leaf': 10}

In [242]:
def train_catboost(X_train, y_train, X_val, y_val, cat_features, params, iterations = 300, eval_metric = 'RMSE', random_state = 7):
  model = CatBoostRegressor(**params, iterations = iterations, eval_metric = eval_metric, random_state = random_state)
  model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_val, y_val), verbose=True)
  return model

In [243]:
depletion_model = train_catboost(X_train, y_train, X_val, y_val, cat_cols, depletion_model_params, iterations = 1000, eval_metric = 'RMSE', random_state = 7)

0:	learn: 1.0566444	test: 1.0449193	best: 1.0449193 (0)	total: 654ms	remaining: 10m 53s
1:	learn: 0.9547578	test: 0.9359605	best: 0.9359605 (1)	total: 1.15s	remaining: 9m 33s
2:	learn: 0.9166969	test: 0.8935694	best: 0.8935694 (2)	total: 1.69s	remaining: 9m 20s
3:	learn: 0.8905651	test: 0.8678223	best: 0.8678223 (3)	total: 2.1s	remaining: 8m 44s
4:	learn: 0.8791528	test: 0.8571901	best: 0.8571901 (4)	total: 2.64s	remaining: 8m 45s
5:	learn: 0.8698406	test: 0.8474534	best: 0.8474534 (5)	total: 3.13s	remaining: 8m 38s
6:	learn: 0.8590535	test: 0.8344894	best: 0.8344894 (6)	total: 3.6s	remaining: 8m 30s
7:	learn: 0.8502988	test: 0.8254432	best: 0.8254432 (7)	total: 3.98s	remaining: 8m 13s
8:	learn: 0.8328921	test: 0.8077832	best: 0.8077832 (8)	total: 4.47s	remaining: 8m 12s
9:	learn: 0.8282987	test: 0.8023407	best: 0.8023407 (9)	total: 4.83s	remaining: 7m 58s
10:	learn: 0.8242206	test: 0.7979607	best: 0.7979607 (10)	total: 5.16s	remaining: 7m 44s
11:	learn: 0.8155785	test: 0.7875490	best:

In [244]:
np.expm1(y_val)[:30].values

array([ 0.        ,  0.        ,  1.45113923,  6.22337397,  0.        ,
       30.        , 10.06535491, 30.        , 12.        ,  3.71070405,
        0.10978186,  0.        ,  0.2842336 ,  0.        , 19.56231412,
        1.06201226,  0.        , 24.        ,  0.        ,  5.82462459,
       14.41090517, 12.9494687 ,  8.87183027,  2.61635315, 11.88285856,
       21.37915536, 14.63315002,  0.25650396,  0.        , 24.        ])

In [245]:
np.expm1(depletion_model.predict(X_val))[:30]

array([ 1.37711891e-01, -6.51894892e-02,  3.01166775e+00,  1.16346910e+01,
        3.33852125e-01,  1.89885916e+01,  1.13854420e+01,  1.06366007e+01,
        1.38321810e+01,  1.51514224e+00,  4.78292438e-01, -6.21140154e-03,
        4.67045008e-01, -4.32336586e-02,  2.10950364e+01,  2.57250551e+00,
        1.07156808e-01,  3.39947995e+01, -2.97265361e-01,  1.64546515e+01,
        1.58073315e+01,  1.53483543e+01,  2.04435143e+00,  1.98163611e+00,
        2.28358830e+01,  2.03134886e+01,  9.16728823e+00,  1.10283025e-01,
       -1.18414169e-01,  2.59870761e+01])

In [246]:
repeated_inv.sample()[train_cols].values

array([['68356326', 'buavita', 'beverages', 'health_drinks',
        'unilever_indonesia', 122.4812383369828, 20.0, 'excess',
        'friends/family']], dtype=object)

In [247]:
# ['G3864100', "maybelline", 'makeup', 'lip_makeup', "l'oreal_malaysia",	600, 90, 'excess', 'liquidator']
# ['62685401', 'rinso', 'home_care', 'laundry','unilever_indonesia', 400, 90,'excess', 'retailer']
# ['G3697100', "l'oreal_paris", 'makeup', 'lip_makeup',"l'oreal_malaysia", 600, 90,'excess', 'liquidator']

In [254]:
# depletion model prompt
test_data_point = ['G3697100', "l'oreal_paris", 'makeup', 'lip_makeup',
        "l'oreal_malaysia", 400, 10,
        'excess', 'liquidator']
print(test_data_point, '\n')
np.expm1(depletion_model.predict(test_data_point))

['G3697100', "l'oreal_paris", 'makeup', 'lip_makeup', "l'oreal_malaysia", 400, 10, 'excess', 'liquidator'] 



6.81069218994782

In [259]:
# def dump_files_for_inference_pipeline():
#   print('Dumping files for inference later ...')
#   joblib.dump(depletion_model, '/content/drive/MyDrive/PSI_Modeling_Data/depletion_model/depletion_model.pkl')

In [258]:
# dump_files_for_inference_pipeline()